In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from tqdm import tqdm_notebook
import pickle
import os
import logging
import time
from IPython.core.debugger import set_trace

import torch
import torch.nn as nn
import torch.nn.functional as F

from dataset_unet import prepare_trainset
from utils import save_checkpoint, load_checkpoint, set_logger
from gpu_utils import set_n_get_device

#from model.model_unet import UNetResNet34, predict_proba
#from model.model_unet_classify_zero import UNetResNet34 as ZeroMaskClassifier
from model.deeplab_model_kaggler.deeplab import DeepLab

%matplotlib inline

In [ ]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'drn' #'xception', 'resnet', 'drn'
#AUX_LOGITS = True#False, only for 'INCEPTION_V3'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0,1,2,3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = [0,1,2,3] #None#[0, 1]#use 2 gpus

SEED = 1234 #5678#4567#3456#2345#1234
debug = True# if True, load 100 samples
IMG_SIZE = 768 #768#1024#512
BATCH_SIZE = 8
NUM_WORKERS = 24
torch.cuda.manual_seed_all(SEED)

## the dataset

In [ ]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

In [ ]:
for i, (images, masks) in enumerate(train_dl):
    images = images.to(device=device, dtype=torch.float)
    masks = masks.to(device=device, dtype=torch.float)
    #labels = (torch.sum(masks.reshape(masks.size()[0], -1), dim=1, keepdim=True)==0).to(device=device, dtype=torch.float) #1 for non-zero-mask
    if i==0:
        break

In [ ]:
images.size(), masks.size()#, labels.size()

## the model

In [ ]:
net = DeepLab(num_classes=2,
              backbone=MODEL,#resnet34, resnet101
              output_stride=16,#default 16, 8
              sync_bn=None,
              freeze_bn=False,
              debug=True
             ).cuda(device=device)

#checkpoint_path = 'checkpoint/UNetResNet34_512_v1_seed3456/best.pth.tar'
#net, _ = load_checkpoint(checkpoint_path, net)

if multi_gpu is not None:
    net = nn.DataParallel(net, device_ids=multi_gpu)

In [ ]:
logit = net(images)

In [ ]:
logit.size(), masks.size()

In [ ]:
# Loss_FUNC = nn.BCEWithLogitsLoss()
# bce_loss = Loss_FUNC(logit, masks)
# bce_loss

In [ ]:
_loss = net.criterion(logit, masks)
_loss

In [ ]:
_metric = net.metric(logit, masks)
_metric

## predict the validset, and analyse

In [ ]:
#move checkpoint from gamma machine to here
cd checkpoint
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/checkpoint/deeplabv3plus_resnet_1280_v2_seed2345/ deeplabv3plus_resnet_1280_v2_seed2345
cd logging
scp -r endi.niu@10.171.36.214:/home/endi.niu/SIIM/logging/deeplabv3plus_resnet_1280_v2_seed2345.log deeplabv3plus_resnet_1280_v2_seed2345.log


In [1]:
import numpy as np
import pandas as pd
import math
from matplotlib import pyplot as plt
from tqdm import tqdm, tqdm_notebook
import pickle
import os
import logging
import time
import gc
from IPython.core.debugger import set_trace

import torch
import torch.nn as nn
import torch.nn.functional as F

from utils import save_checkpoint, load_checkpoint, set_logger
from gpu_utils import set_n_get_device

from model.deeplab_model_kaggler.deeplab import DeepLab, predict_proba
from dataset_unet import prepare_trainset

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def inverse_sigmoid(x):
    return np.log(x / (1-x))

%matplotlib inline

In [2]:
######### Config the training process #########
#device = set_n_get_device("0, 1, 2, 3", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
MODEL = 'resnet'#
#AUX_LOGITS = True#False, only for 'INCEPTION_V3'
print('====MODEL ACHITECTURE: %s===='%MODEL)

device = set_n_get_device("0,1", data_device_id="cuda:0")#0, 1, 2, 3, IMPORTANT: data_device_id is set to free gpu for storing the model, e.g."cuda:1"
multi_gpu = [0,1] #None#[0, 1]#use 2 gpus

SEED = 2345 #5678#4567#3456#2345#1234
debug = False# if True, load 100 samples
IMG_SIZE = 1024 #768#1024#512
BATCH_SIZE = 32 #32#16
NUM_WORKERS = 24
torch.cuda.manual_seed_all(SEED)

====MODEL ACHITECTURE: resnet====


In [3]:
train_dl, val_dl = prepare_trainset(BATCH_SIZE, NUM_WORKERS, SEED, IMG_SIZE, debug)

Count of trainset (for training):  9607
Count of validset (for training):  1068


In [ ]:
# y should be makeup
y_valid = []
for i, (image, masks) in enumerate(val_dl):
    #if i==10:
    #    break
    truth = masks.to(device=device, dtype=torch.float)
    y_valid.append(truth.cpu().numpy())
y_valid = np.concatenate(y_valid, axis=0)
y_valid.shape

In [ ]:
net = DeepLab(num_classes=2,
              backbone=MODEL,#resnet34, resnet101
              output_stride=16,#default 16, 8
              sync_bn=None,
              freeze_bn=False,
              debug=False
             ).cuda(device=device)

#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v1_seed1234/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v1_seed3456/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_768_v1_seed3456/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_768_v1_seed2345/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_768_v1_seed1234/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v1_seed2345/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_768_v1_seed4567/best.pth.tar'

#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed1234/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed2345/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed3456/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed5678/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed6789/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed7890/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed8901/best.pth.tar'
#checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1024_v2_seed9012/best.pth.tar'
checkpoint_path = 'checkpoint/deeplabv3plus_resnet_1280_v2_seed2345/best.pth.tar'

net, _ = load_checkpoint(checkpoint_path, net)

if multi_gpu is not None:
    net = nn.DataParallel(net, device_ids=multi_gpu)

In [ ]:
%%time
preds_valid = predict_proba(net, val_dl, device, multi_gpu=multi_gpu, mode='valid', tta=True)

In [ ]:
y_valid.shape, preds_valid.shape

In [ ]:
## search for best thresholds
def calculate_dice(logit, truth, EMPTY_THRESHOLD=400, MASK_THRESHOLD=0.22):
    IMG_SIZE = logit.shape[-1] #256
    logit = sigmoid(logit)#.reshape(n, -1)
    pred = (logit>MASK_THRESHOLD).astype(np.int)
    pred_clf = (pred.reshape(pred.shape[0], -1).sum(axis=1)<EMPTY_THRESHOLD).astype(np.int)
    pred[pred_clf.reshape(-1,)==1, ] = 0
    return dice_overall(pred, truth)

def dice_overall(pred_mask, truth_mask, eps=1e-8):
    n = pred_mask.shape[0]
    pred_mask = pred_mask.reshape(n, -1)
    truth_mask = truth_mask.reshape(n, -1)
    intersect = (pred_mask * truth_mask).sum(axis=1).astype(np.float)
    union = (pred_mask + truth_mask).sum(axis=1).astype(np.float)
    return ((2.0*intersect + eps) / (union+eps)).mean()

In [ ]:
#np.arange(1400, 1520, 20) for 512#np.arange(350, 450, 10) for 256
#EMPTY_THRESHOLD_candidate = np.arange(6000, 7000, 100) #for 1024
EMPTY_THRESHOLD_candidate = np.arange(9500, 10500, 100) #for 1280
#EMPTY_THRESHOLD_candidate = np.arange(2900, 3300, 100)#np.arange(2900, 4200, 100)#for 768
MASK_THRESHOLD_candidate = np.arange(0.18, 0.23, 0.01)#np.arange(0.19, 0.27, 0.01)
M, N = len(EMPTY_THRESHOLD_candidate), len(MASK_THRESHOLD_candidate)
best_threshold = None
best_score = 0

for i in tqdm_notebook(range(M)):
    EMPTY_THRESHOLD = EMPTY_THRESHOLD_candidate[i]
    for j in range(N):
        MASK_THRESHOLD = MASK_THRESHOLD_candidate[j]
        dice_score = calculate_dice(preds_valid, y_valid.squeeze(1), EMPTY_THRESHOLD, MASK_THRESHOLD)
        print('CLF_EMPTY_THRESHOLD: %f, MASK_THRESHOLD: %f, dice_score: %f'%(EMPTY_THRESHOLD, MASK_THRESHOLD, dice_score))
        if dice_score>best_score:
            best_threshold = [EMPTY_THRESHOLD, MASK_THRESHOLD]
            best_score = dice_score

In [ ]:
EMPTY_THRESHOLD, MASK_THRESHOLD = best_threshold
#EMPTY_THRESHOLD, MASK_THRESHOLD, best_score = 6000, 0.21, 0.855777
EMPTY_THRESHOLD, MASK_THRESHOLD, best_score

In [ ]:
def predict_mask(logit, EMPTY_THRESHOLD, MASK_THRESHOLD):
    """Transform each prediction into mask.
    input shape: (256, 256)
    """
    #pred mask 0-1 pixel-wise
    #n = logit.shape[0]
    IMG_SIZE = logit.shape[-1] #256
    #EMPTY_THRESHOLD = 100.0*(IMG_SIZE/128.0)**2 #count of predicted mask pixles<threshold, predict as empty mask image
    #MASK_THRESHOLD = 0.22
    #logit = torch.sigmoid(torch.from_numpy(logit)).view(n, -1)
    #pred = (logit>MASK_THRESHOLD).long()
    #pred[pred.sum(dim=1) < EMPTY_THRESHOLD, ] = 0 #bug here, found it, the bug is input shape is (256, 256) not (16,256,256)
    logit = sigmoid(logit)#.reshape(n, -1)
    pred = (logit>MASK_THRESHOLD).astype(np.int)
    if pred.sum() < EMPTY_THRESHOLD:
        return np.zeros(pred.shape).astype(np.int)
    else:
        return pred
    return pred

it seems like ignoring EMPTY_THRESHOLD gives higher recall, although mask is low quality

In [ ]:
## visualize predicted masks
start = 0
rows = 10

cnt = 0
for idx, (img, mask) in enumerate(val_dl):
    if idx<start:
        continue
    for j in range(BATCH_SIZE):#BATCH_SIZE=8
        not_empty = mask[j][0].sum()>0
        if not_empty:
            cnt+=1
            pred_mask = predict_mask(preds_valid[idx*BATCH_SIZE+j], EMPTY_THRESHOLD, MASK_THRESHOLD)#EMPTY_THRESHOLD=0
            #if pred_mask.sum()==0:
            #    continue
            fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
            ax0.imshow(img[j][0].numpy(), plt.cm.bone)
            ax1.imshow(mask[j][0], vmin=0, vmax=1, cmap="Reds")
            ax2.imshow(pred_mask, vmin=0, vmax=1, cmap="Blues")
            if not_empty.item():
                ax1.set_title('Targets(Has Mask)')
            else:
                ax1.set_title('Targets(Empty)')
            ax2.set_title('Predictions')
        if cnt>rows:
            break
    if cnt>rows:
            break

In [ ]:
# ## visualize predicted masks
# rows = 20

# cnt = 0
# for idx, (img, mask) in enumerate(val_dl):
#     for j in range(BATCH_SIZE):#BATCH_SIZE=8
#         is_empty = mask[j][0].sum()==0
#         if is_empty:
#             cnt+=1
#             pred_mask = predict_mask(preds_valid[idx*BATCH_SIZE+j], EMPTY_THRESHOLD, MASK_THRESHOLD)
#             #if pred_mask.sum()==0:
#             #    continue
#             fig, (ax0, ax1, ax2) = plt.subplots(ncols=3, figsize=(12, 4))
#             ax0.imshow(img[j][0].numpy(), plt.cm.bone)
#             ax1.imshow(mask[j][0], vmin=0, vmax=1, cmap="Reds")
#             ax2.imshow(pred_mask, vmin=0, vmax=1, cmap="Blues")
#             if is_empty.item():
#                 ax1.set_title('Targets(Empty Mask)')
#             else:
#                 ax1.set_title('Targets(Has Mask)')
#             ax2.set_title('Predictions')
#         if cnt>rows:
#             break
#     if cnt>rows:
#             break

In [ ]:
# s = (sigmoid(preds_valid)>MASK_THRESHOLD).reshape(1064, -1).sum(axis=1)
# (s>1420).mean(), (s>0).mean()

## predict the testset

In [4]:
import glob
from dataset_unet import prepare_testset

In [5]:
test_fnames = [f.split('/')[-1][:-4] for f in glob.glob('data/processed/test/*')]
len(test_fnames), test_fnames[0]

(1377, '1.2.276.0.7230010.3.1.4.8323329.6160.1517875196.806852')

In [6]:
test_dl = prepare_testset(BATCH_SIZE, NUM_WORKERS, IMG_SIZE)

In [ ]:
%%time
preds_test = predict_proba(net, test_dl, device, multi_gpu=multi_gpu, mode='test', tta=True)

In [ ]:
preds_test.shape

In [ ]:
## visualize predicted masks
start = 0
total = 19

fig=plt.figure(figsize=(15, 20))
cnt = 0
for idx, img in enumerate(test_dl):
    if idx<start:
        continue
    for j in range(BATCH_SIZE):#BATCH_SIZE=8
        cnt+=1
        pred_mask = predict_mask(preds_test[idx*BATCH_SIZE+j], EMPTY_THRESHOLD, MASK_THRESHOLD)
        #if pred_mask.float().mean()==0:
        #    continue
        ax = fig.add_subplot(5, 4, cnt)
        plt.imshow(img[j][0].numpy(), plt.cm.bone)
        plt.imshow(pred_mask, alpha=0.3, cmap="Reds")
        if pred_mask.sum()>0:
            plt.title('Predict Mask')
        else:
            plt.title('Predict Empty')
        if cnt>total:
            break
    if cnt>total:
            break

## build submission

In [ ]:
import PIL
from mask_functions import mask2rle

In [ ]:
%%time
#### Step 1: Generate rle encodings (images are first converted to the original size)
rles = []
for p in tqdm_notebook(preds_test):#p is logit from model
    pred_mask = predict_mask(p, EMPTY_THRESHOLD, MASK_THRESHOLD)
    if pred_mask.sum()>0:#predicted non-empty mask
        im = PIL.Image.fromarray((pred_mask.T*255).astype(np.uint8)).resize((1024,1024))
        im = np.asarray(im)
        rles.append(mask2rle(im, 1024, 1024))
    else: rles.append('-1')
    
sub_df = pd.DataFrame({'ImageId': test_fnames, 'EncodedPixels': rles})
print(len(sub_df.index))
sub_df.head()

In [ ]:
sub_df.head()

In [ ]:
# check the correctness of transformation
pred_mask = predict_mask(preds_test[22], EMPTY_THRESHOLD, MASK_THRESHOLD)
im = PIL.Image.fromarray((pred_mask.T*255).astype(np.uint8)).resize((1024,1024))
im = np.asarray(im)
im.max()

In [ ]:
plt.imshow(im)

In [ ]:
sub_df.to_csv('submission/0826_deeplabv3plus_1280_seed2345_tta_v2_10300_023.csv.gz', index=False, compression='gzip')

In [ ]:
(sub_df.EncodedPixels!='-1').mean()

In [ ]:
#this is for remembering best thresholds of those already submitted solutions
sub_df.to_csv('submission/del.csv.gz', index=False, compression='gzip')

In [ ]:
EMPTY_THRESHOLD, MASK_THRESHOLD

In [ ]:
rd_mask = np.zeros((1024, 1024))
d = 50
rd_mask[400:400+d, 400:400+d] = 1.0

In [ ]:
plt.imshow(im, cmap=plt.cm.bone)
plt.imshow(rd_mask, alpha=0.25, cmap='Reds')